# Planner Cost Gap CDF
Compute the relative cost gap between the dynamic programming planner and an exhaustive oracle for a small set of circuits and plot the cumulative distribution function (CDF).

In [ ]:
from functools import lru_cache
import numpy as np
import matplotlib.pyplot as plt
from benchmarks.circuits import ghz_circuit, qft_circuit, qft_on_ghz_circuit, w_state_circuit, grover_circuit
from quasar.circuit import Circuit
from quasar.planner import Planner, _simulation_cost, _supported_backends


In [ ]:
def cost_gap(circuit: Circuit) -> float:
    planner = Planner()
    gates = circuit.gates
    res = planner.plan(circuit)
    dp_cost = res.table[-1][res.final_backend].cost.time

    @lru_cache(None)
    def dfs(i: int) -> float:
        if i >= len(gates):
            return 0.0
        best = float("inf")
        for j in range(i + 1, len(gates) + 1):
            seg = gates[i:j]
            allowed = _supported_backends(
                seg,
                allow_tableau=True,
                estimator=planner.estimator,
                sparsity=circuit.sparsity,
                phase_rotation_diversity=circuit.phase_rotation_diversity,
                amplitude_rotation_diversity=circuit.amplitude_rotation_diversity,
            )
            nq = len({q for g in seg for q in g.qubits})
            ng = len(seg)
            nm = sum(1 for g in seg if g.gate.upper() in {"MEASURE", "RESET"})
            n1 = sum(1 for g in seg if len(g.qubits) == 1 and g.gate.upper() not in {"MEASURE", "RESET"})
            n2 = ng - n1 - nm
            for backend in allowed:
                cost = _simulation_cost(planner.estimator, backend, nq, n1, n2, nm).time
                best = min(best, cost + dfs(j))
        return best

    oracle_cost = dfs(0)
    return (dp_cost - oracle_cost) / oracle_cost


In [ ]:
circuits = {
    "ghz4": ghz_circuit(4),
    "qft4": qft_circuit(4),
    "qft_on_ghz4": qft_on_ghz_circuit(4),
    "w4": w_state_circuit(4),
    "grover4": grover_circuit(4, 1),
}

gaps = np.array([cost_gap(c) for c in circuits.values()])
cdf_x = np.sort(gaps)
cdf_y = np.linspace(0, 1, len(cdf_x), endpoint=True)

plt.plot(cdf_x, cdf_y, marker="o")
plt.xlabel("Relative cost gap (DP vs oracle)")
plt.ylabel("CDF")
plt.title("Planner Cost Gap CDF")
plt.grid(True)
plt.show()
